In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error

In [ ]:
# Load the dataset
data = pd.read_csv('sales_timeseries.csv')

# Data preparation
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

# Time Series Visualization
data['TotalSales'].plot(title='Daily Total Sales', figsize=(12, 6))

# Moving Average
data['MA_7'] = data['TotalSales'].rolling(window=7).mean()
data[['TotalSales', 'MA_7']].plot(title='7-day Moving Average', figsize=(12, 6))

# Seasonal Decomposition
result = seasonal_decompose(data['TotalSales'], model='additive', freq=365)
result.plot()
plt.show()

# Statistical Test for Stationarity
result_adf = adfuller(data['TotalSales'])
print('ADF Statistic: %f' % result_adf[0])
print('p-value: %f' % result_adf[1])

# ARIMA Forecasting
model = ARIMA(data['TotalSales'], order=(5,1,0))
model_fit = model.fit(disp=0)
forecast = model_fit.forecast(steps=30)[0]

# Plot Forecast
plt.plot(forecast)
plt.title('Sales Forecast')
plt.show()

# The code for calculating the forecast accuracy is omitted for brevity
